# Imports

In [ ]:
import itertools
from collections import Counter
import string
import numpy as np
from tqdm.notebook  import tqdm
import sys
import time
import re

# Load text file

In [ ]:
characters = set()
words = []
limit = int(1e6)

text = open("list/games.txt", "rb").read().decode("utf8").lower()
unprocessed_words = re.split("\s", text)[0:limit]
for word in unprocessed_words:
    if len(word) < 4:
        continue
    if any([i not in string.ascii_lowercase for i in word]):
        continue
    characters |= set(word)
    words.append(word)

characters = sorted(list(characters))
print(f"characters : {len(characters)}")
print(characters)
print(f"nb words : {len(words)}")
print(words[0:10])

# N-grams

In [ ]:
def n_grams(word, n):
    return [word[i:i+n] for i in range(0, len(word) - n + 1)]
n_grams(words[5], 3)

# Probability tables

In [ ]:
def create_probability_table(chain):
    counter = Counter(chain)
    p = np.zeros(len(counter.most_common(1)[0][0]) * [len(characters)])
    for w in counter:
        pos = tuple([characters.index(i) for i in w])
        p[pos] = counter[w]
    return p

chain_s1 = itertools.chain(word[:1] for word in tqdm(words))
chain_s2 = itertools.chain(word[:2] for word in tqdm(words))
chain_s3 = itertools.chain(word[:3] for word in tqdm(words))
chain_m4 = itertools.chain.from_iterable(n_grams(word, 4) for word in tqdm(words))
chain_e3 = itertools.chain(word[-3:] for word in tqdm(words))
chain_e2 = itertools.chain(word[-2:] for word in tqdm(words))

ps1 = create_probability_table(chain_s1)
ps2 = create_probability_table(chain_s2)
ps3 = create_probability_table(chain_s3)
pm4 = create_probability_table(chain_m4)
pe3 = create_probability_table(chain_e3)
pe2 = create_probability_table(chain_e2)

In [ ]:
def print_stats(p):
    a = p.reshape(-1)
    print(f"{np.sum(a!=0)}/{a.shape[0]}")
    
print_stats(ps1)
print_stats(ps2)
print_stats(ps3)
print_stats(pm4)
print_stats(pe3)
print_stats(pe2)

# Generative model

In [ ]:
def generate(n, s="", laplace=1e-1):
    s = s.lower()
    for i in range(n):
        # get the previous characters indices
        if len(s) >= 1:
            n_minus_1 = characters.index(s[len(s)-1])
        if len(s) >= 2:
            n_minus_2 = characters.index(s[len(s)-2])
        if len(s) >= 2:
            n_minus_3 = characters.index(s[len(s)-3])
        
        # select the probability table
        
        # begining of the word
        if len(s) <= 0:
            p = ps1
        elif len(s) <= 1:
            p = ps2[n_minus_1, :]
        elif len(s) <= 2:
            p = ps3[n_minus_2, n_minus_1, :]
        # end of the word
        elif len(s) >= n - 1:
            p = pe2[n_minus_1, :]
        elif len(s) >= n - 2:
            p = pe3[n_minus_2, n_minus_1, :]
        # middle of the word
        else:
            p = pm4[n_minus_3, n_minus_2, n_minus_1, :]

        # laplace smoothing
        p = p.copy()
        p += laplace
        for i in range(10):
            p = p / p.sum()
        
        # select the next character
        s += np.random.choice(characters, p=p)
    return s

In [ ]:
def compute_probability(w):
    def prob(p, c, laplace=1e-1):
        sum_ = p.sum() + laplace
        qte = p[c] + len(p.reshape(-1)) * laplace
        return np.log(qte / sum_)
    
    p = 0
    
    characters_indices = [characters.index(wi) for wi in w]
    
    # start letters probability
    if len(characters_indices) > 0:
        p += prob(ps1, (characters_indices[0],))
    
    if len(characters_indices) > 1:
        p += prob(ps2, (characters_indices[0], characters_indices[1]))
        
    if len(characters_indices) > 2:
        p += prob(ps3, (characters_indices[0], characters_indices[1], characters_indices[2]))
        
    # end letters probability
    if len(characters_indices) > 1:
        p += prob(pe2, (characters_indices[-2], characters_indices[-1],))
    
    if len(characters_indices) > 2:
        p += prob(pe3, (characters_indices[-3], characters_indices[-2], characters_indices[-1],))
        
    # in-between probability
    if len(characters_indices) > 3:
        for ch in [tuple(characters_indices[i:i+4]) for i in range(len(characters_indices)-3)]:
            p += prob(pm4, ch)
        
    return p

# Use cases

## Generate words based on this word list

In [ ]:
for i in range(20):
    print(generate(int(np.random.randint(4, 15))))

## Generate words starting by x

In [ ]:
for i in range(20):
    print(generate(2, "zeph").title())

## Find anagram of a word and sort them by their probability of occuring in the language

In [ ]:
word = "raphael"
perms = set("".join(ws) for ws in itertools.permutations(word, len(word)))
probabilities = [(p, compute_probability(p)) for p in tqdm(perms)]
probabilities.sort(key=lambda x: -x[-1])
for p in probabilities[0:20]:
    print(p[0])

## Generate a lot of words in a file

In [ ]:
ws = [generate(8) for w in tqdm(range(10000))]
open("generated/words.txt", "w").write("\r\n".join(ws))